## LSA(Latent Semantic Analysis, 잠재 의미 분석)
- 문서의 잠재된 의미 분석 및 단어에 잠재된 의미 분석 가능
- 의미는 문서와 단어를 연결하는 매개체, 축소된 차원이 이 역할?
- 절단된 SVD(Truncated SVD)로 구현됨
<br/>

**SVD(Singular Value Decomposition):** 특이값 분해, m x n 크기 행렬을 세 개 행렬의 곱으로 분해하는 것 <br/><br/>
$$ X = UΣV^T $$ <br/>
``U와 V: m x m, n x n 크기를 갖는 직교행렬 
Σ: m x n 크기의 대각행렬 ``

분해된 세 행렬을 다시 곱해 원래 데이터를 복원할 수 있음 <br/>
단, 절단된 SVD에서는 완전한 복원이 불가능, 최대한 유사하게
<br/>

[SVD 관련 링크](https://wikidocs.net/24949)

In [1]:
# 20뉴스그룹 데이터 불러오기
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# train
news_train = fetch_20newsgroups(subset='train',
                                remove=('headers', 'footers', 'quotes'),
                                categories=categories)

# test
news_test = fetch_20newsgroups(subset='test',
                               remove=('headers', 'footers', 'quotes'),
                               categories=categories)

In [2]:
# 전처리
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

cachedStopWords = stopwords.words('english')   # 불용어

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

# train/test split
X_train = news_train.data
y_train = news_train.target

X_test = news_test.data
y_test = news_test.target

# 토큰화
reg = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english'))

In [3]:
# pca에서 토크나이저 임포트
import import_ipynb
from pca import tokenizer

# tfidf
tfidf = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

importing Jupyter notebook from pca.ipynb
# Train score: 0.962
# Test score: 0.761
Original Tfidf matrix shape:  (2034, 20085)
PCA Converted matrix shape:  (2034, 2000)
Sum of explained variance ratio: 1.000
# Train score: 0.962
# Test score: 0.761
# Train score: 0.790
# Test score: 0.718
# Used features: 321 out of  (2034, 20085)
PCA Converted matrix shape:  (2034, 321)
Sum of explained variance ratio: 0.437
# Train score: 0.875
# Test score: 0.751


In [4]:
# LSA
from sklearn.decomposition import TruncatedSVD

# pca과 마찬가지로 차원의 개수 2000개로 지정
svd = TruncatedSVD(n_components=2000, random_state=7)
X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)

print("LSA Converted X shape: ", X_train_lsa.shape)

print("Sum of explained variance ratio: {:.3f}".format(svd.explained_variance_ratio_.sum()))

LSA Converted X shape:  (2034, 2000)
Sum of explained variance ratio: 1.000


In [5]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_lsa, y_train)

print("# Train score: {:.3f}".format(lr.score(X_train_lsa, y_train)))
print("# Test score: {:.3f}".format(lr.score(X_test_lsa, y_test)))

# Train score: 0.962
# Test score: 0.761


In [6]:
# 100개 차원으로 축소
svd = TruncatedSVD(n_components=100, random_state=1)
X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)

print("LSA Converted X shape: ", X_train_lsa.shape)

print("Sum of explained variance ratio: {:.3f}".format(svd.explained_variance_ratio_.sum()))

lr.fit(X_train_lsa, y_train)

print("# Train score: {:.3f}".format(lr.score(X_train_lsa, y_train)))
print("# Test score: {:.3f}".format(lr.score(X_test_lsa, y_test)))

LSA Converted X shape:  (2034, 100)
Sum of explained variance ratio: 0.209
# Train score: 0.810
# Test score: 0.745


### LSA를 이용한 의미 기반 유사도 계산

LSA에서 축소된 차원은 잠재된 의미를 보여준다고 할 수 있음 <br/>
앞서 4장에서 했던 것처럼 문서를 벡터로 변환한 뒤 유사도 계산하는 것이 가능 <br/>

In [9]:
# 코사인 유사도
# 첫번째 문서와 전체 문서와의 유사도를 계산

from sklearn.metrics.pairwise import cosine_similarity

print("전체 카테고리: ", news_train.target_names)
print("첫 문서 카테고리: ", y_train[0])


sim = cosine_similarity([X_train_lsa[0]], X_train_lsa)

# 유사도가 높은 30개 문서의 인덱스
print("Top 20 유사도(lsa):\n", sorted(sim[0].round(2), reverse=True)[:30])
sim_index = (-sim[0]).argsort()[:30]
print("Top 20 유사도의 인덱스(lsa):\n", sim_index)
sim_labels = [y_train[i] for i in sim_index]
print("Top 20 유사 뉴스의 카테고리(lsa):\n", sim_labels)

# 1개 문서를 제외한 모든 문서가 첫번째 문서와 같은 카테고리(comp.graphics)임을 알 수 있음

전체 카테고리:  ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
첫 문서 카테고리:  1
Top 20 유사도(lsa):
 [1.0, 0.74, 0.74, 0.72, 0.7, 0.7, 0.69, 0.67, 0.66, 0.65, 0.65, 0.65, 0.63, 0.62, 0.62, 0.62, 0.57, 0.57, 0.55, 0.54, 0.54, 0.54, 0.53, 0.53, 0.52, 0.5, 0.5, 0.48, 0.48, 0.48]
Top 20 유사도의 인덱스(lsa):
 [   0 1957 1674  501 1995 1490  790 1902 1575 1209 1728  892 1892  998
 1038 1826 1290 1089  867  151 1691 1029   25  651  783  864  874  897
 1724  942]
Top 20 유사 뉴스의 카테고리(lsa):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
# tfidf
sim = cosine_similarity(X_train_tfidf[0], X_train_tfidf)

# 유사도가 높은 30개 문서의 인덱스
print("Top 20 유사도(tfidf):\n", sorted(sim[0].round(2), reverse=True)[:30])    # 내림차순
sim_index = (-sim[0]).argsort()[:30]     # argsort(): 배열을 정렬하는 인덱스 출력, 여기서는 내림차순 인덱스를 출력
print("Top 20 유사도의 인덱스(tfidf):\n", sim_index)
sim_labels = [y_train[i] for i in sim_index]
print("Top 20 유사 뉴스의 카테고리(tfidf):\n", sim_labels)

# 유사도 값이 lsa보다 낮음을 알 수 있음
# -> 차원이 lsa보다 상대적으로 커서 차원의 저주가 적용됐기 때문

Top 20 유사도(tfidf):
 [1.0, 0.3, 0.22, 0.21, 0.19, 0.19, 0.19, 0.17, 0.16, 0.16, 0.16, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.13, 0.13, 0.13, 0.13, 0.13]
Top 20 유사도의 인덱스(tfidf):
 [   0 1575 1892 1490  501 1290 1013  998 1636 1705 1995 1957 1664  651
 1038  429 1089 1209 1728 1803 1724  783  867 1578 1902  169  715  499
  697  537]
Top 20 유사 뉴스의 카테고리(tfidf):
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]


### 토픽 모델링
잠재된 의미와 연결된 단어들을 통해 문서를 이루고 있는 잠재 의미를 파악할 수 있음 <br/>
이런 잠재 의미를 '토픽'이라고 하고, 분석 방법을 토픽 모델링이라고 함 <br/> <br/>
최근의 토픽 모델링은 대부분 LDA(Latent Dirichlet Allocation)을 사용하지만 <br/>
LDA가 나오기 전에는 LSA가 토픽 모델링에 사용되기도 했음

In [11]:
# 차원을 10개로 축소
svd = TruncatedSVD(n_components=10, random_state=1)
X_train_lsa = svd.fit_transform(X_train_tfidf)
X_test_lsa = svd.transform(X_test_tfidf)

print("LSA Converted X shape: ", X_train_lsa.shape)

print("Sum of explained variance ratio: {:.3f}".format(svd.explained_variance_ratio_.sum()))

LSA Converted X shape:  (2034, 10)
Sum of explained variance ratio: 0.045


In [15]:
# 토픽별로 비중이 높은 단어 10개 출력
terms = tfidf.get_feature_names_out()

def get_topics(model, feature_names, n=10):
    for idx, topic in enumerate(model.components_):   # components_: 각 단어에 대해 축소된 차원, 즉 의미의 비중
        print("Topic %d: "%(idx+1), [feature_names[i] for i in topic.argsort()[:-n-1:-1]], # 처음부터 끝까지 내림차순으로(-1) 가져오기
              [topic[i].round(2) for i in topic.argsort()[:-n-1:-1]])    # topic
get_topics(svd, terms)

Topic 1:  ['would', 'one', 'god', 'think', 'use', 'peopl', 'know', 'like', 'say', 'space'] [0.17, 0.15, 0.14, 0.13, 0.13, 0.13, 0.12, 0.12, 0.11, 0.11]
Topic 2:  ['file', 'imag', 'thank', 'program', 'graphic', 'space', 'format', 'use', 'color', 'ftp'] [0.22, 0.18, 0.17, 0.16, 0.16, 0.15, 0.12, 0.1, 0.1, 0.1]
Topic 3:  ['space', 'orbit', 'nasa', 'launch', 'shuttl', 'satellit', 'year', 'moon', 'lunar', 'cost'] [0.4, 0.17, 0.17, 0.17, 0.1, 0.1, 0.1, 0.1, 0.1, 0.09]
Topic 4:  ['moral', 'object', 'system', 'valu', 'goal', 'think', 'anim', 'absolut', 'natur', 'defin'] [0.53, 0.34, 0.2, 0.12, 0.1, 0.1, 0.09, 0.08, 0.08, 0.07]
Topic 5:  ['ico', 'bobb', 'tek', 'beauchain', 'bronx', 'manhattan', 'sank', 'queen', 'vice', 'blew'] [0.23, 0.23, 0.23, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22]
Topic 6:  ['god', 'file', 'imag', 'object', 'moral', 'exist', 'space', 'format', 'system', 'color'] [0.36, 0.23, 0.23, 0.17, 0.16, 0.14, 0.14, 0.13, 0.1, 0.08]
Topic 7:  ['file', 'islam', 'imag', 'cview', 'use',